# Day 19, robotic pathfinding

This year's prevailing puzzle theme appears to be pathfinding! This is very much like [this year's day 16 puzzle](./Day%2016.ipynb), in that there is a limited time to 'open' resource-producing robots, and crack open the most geodes.

Instead of you moving around a set of tunnels and opening valves one by one, we have robots bring resources to a single robot factory, so that's going to be the focus of a BFS traversal. The factory needs resources produced by robots, but it can only use a certain amount of those resources when it works, putting an upper limit on the number of robots you'd want to output (e.g. in the first blueprint in the example, the factory will need, at most, 4 ore for a robot type, so we can avoid wasting time building more than 3 additional ore-producing robots to add to your starter robot).

One thing to realise is that there is a theoretical upper limit on the number of geodes that can be cracked in the time given: if you had the resources to produce a new geode-cracking robot every minute, then the maximum number of geodes produced by the robots that'll be built in time $T$ is the [triangle number of $T - 1$](https://en.wikipedia.org/wiki/Triangular_number), so $\frac {T(T - 1)} {2}$ (the robot you produce _this_ minute starts outputing geodes the next minute, hence $T - 1$). That property can be used to prune the search tree further; if a given state will only produce at most *max geodes cracked with existing robots + geode-cracking potential*, and we already have found another state that cracks more geodes, we can discard this specific path.

In [1]:
# pyright: strict
from collections import deque
from dataclasses import dataclass
from enum import IntEnum
from typing import Iterable, Iterator, Literal, NamedTuple, Self, TypeAlias


Amount: TypeAlias = int
# The number of robots we have
Robots: TypeAlias = tuple[Amount, Amount, Amount, Amount]
# The number of resources we can build in the time, with the robots we have
Resources: TypeAlias = tuple[Amount, Amount, Amount, Amount]
# the amount of resources the factory requires to build one robot (the factory
# only needs ore, clay and obsidian).
Recipe: TypeAlias = tuple[Amount, Amount, Amount]


class Resource(IntEnum):
    ore = 0
    clay = 1
    obsidian = 2
    geode = 3


class RobotFactoryState(NamedTuple):
    remaining: int
    robots: Robots = (1, 0, 0, 0)
    resources: Resources = (0, 0, 0, 0)

    def traverse(self, bp: "Blueprint") -> Iterator[Self]:
        rem, robots, resources = self.remaining, self.robots, self.resources
        per_robot = zip(Resource, bp.recipes, robots, bp.max_robots, resources)
        # what robot to build next?
        for rtype, recipe, have, rmax, res in per_robot:
            if rmax and (have == rmax or have * rem + res >= rmax * rem):
                # factory can't consume more of this resource, no point in
                # producing this type.
                continue
            if not all(bool(prod) for prod, req in zip(robots, recipe) if req):
                # Not all resources can be produced yet
                continue
            # how much time do we need to produce enough of each required resource?
            needed = 1 + (
                max(
                    0 if avail >= req else (req - avail + prod - 1) // prod
                    for req, avail, prod in zip(recipe, resources, robots)
                    if req
                )
            )
            if needed >= rem:  # no time left to build this robot
                continue
            # produce a new state, with the resources that'll be made available
            # by the time the new robot is done minus the resources consumed by
            # the factory to build the new robot, and the new number of robots
            # with this type incremented.
            new_resources = [
                avail - req + prod * needed
                for avail, req, prod in zip(resources, (*recipe, 0), robots)
            ]
            new_robots = list(robots)
            new_robots[rtype] += 1
            yield RobotFactoryState(
                rem - needed, tuple(new_robots), tuple(new_resources)
            )

    @property
    def max_geodes(self) -> Amount:
        """Max geodes this state can produce given the remaining time and built robots"""
        return (
            self.resources[Resource.geode]
            + self.remaining * self.robots[Resource.geode]
        )

    @property
    def max_geode_potential(self) -> Amount:
        """Max geodes this state can produce given the remaining time, built robots, and potential robots"""
        return self.max_geodes + self.remaining * (self.remaining - 1) // 2


@dataclass(frozen=True)
class Blueprint:
    # the recipe for each robot type
    recipes: tuple[Recipe, Recipe, Recipe, Recipe]
    # the upper limit for each robot, set by the maximum amount of each resource
    # the factory can utilise. (The factory never needs Geode-cracking robots)
    max_robots: tuple[Amount, Amount, Amount, Literal[0]]

    @classmethod
    def from_line(cls, line: str) -> Self:
        # we only need the numbers, that happen to appear on indices that
        # are multiples of 3.
        words = line.split()[6::3]
        recipes = (
            # ore robot, amount of ore
            (int(words[0]), 0, 0),
            # clay robot, amount of ore
            (int(words[2]), 0, 0),
            # obsidian robot, amount of ore and clay
            (int(words[4]), int(words[5]), 0),
            # geode-cracking robot, amount of ore and obsidian
            (int(words[7]), 0, int(words[8])),
        )
        max_robots = [max(resource) for resource in zip(*recipes)]
        return cls(recipes, (*max_robots, 0))  # no limit on the number of geode bots

    def maximum_opened_geodes(self, time: int) -> int:
        start = RobotFactoryState(time)
        queue = deque([start])
        seen = {start}
        max_geodes = 0
        while queue:
            for state in queue.popleft().traverse(self):
                if state in seen or state.max_geode_potential < max_geodes:
                    continue
                max_geodes = max(max_geodes, state.max_geodes)
                seen.add(state)
                queue.append(state)
        return max_geodes


@dataclass
class Factory:
    blueprints: list[Blueprint]

    @classmethod
    def from_lines(cls, lines: Iterable[str]) -> Self:
        return cls([Blueprint.from_line(bp) for bp in lines])

    def quality_levels(self, time: int = 24) -> Iterator[int]:
        for i, bp in enumerate(self.blueprints, 1):
            yield bp.maximum_opened_geodes(time) * i


example = Factory.from_lines(
    # multi-line blueprints still work as we split by variable-length whitespace.
    """\
Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian.
""".split(
        "\n\n"
    )
)

assert sum(example.quality_levels()) == 33

In [2]:
import aocd


factory = Factory.from_lines(aocd.get_data(day=19, year=2022).splitlines())
print("Part 1:", sum(factory.quality_levels()))

Part 1: 1480


## Part 2, run for longer

For part two, we need to extend the runtime of our simulation. Those 8 extra minutes produce a lot more states!

I didn't add any further optimisations, it completes in ~3 seconds time as it stands. The test example is actually worse than my puzzle input, it takes over 6 seconds to verify!

In [3]:
from functools import reduce
from operator import mul


def largest_number_geodes(factory: Factory) -> Iterator[int]:
    for bp in factory.blueprints[:3]: 
        yield bp.maximum_opened_geodes(32)


assert reduce(mul, largest_number_geodes(example))

In [4]:
print("Part 2:", reduce(mul, largest_number_geodes(factory)))

Part 2: 3168
